<a href="https://colab.research.google.com/github/PetraB42/DSpracticum2020/blob/main/E_coli_promoters_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!pip install biopython

In [ ]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Reshaping data from fasta to txt

In [ ]:
classes = ['nonpromoters', 'promoters']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://raw.githubusercontent.com/simecek/dspracticum2020/master/lecture_08/assignment/e_coli_promoters/e_coli_{c}_{s}.fa", f"e_coli_{c}_{s}.fa")

In [ ]:
for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

In [ ]:
for c in classes:
    for s in sets:
        with open(f"e_coli_{c}_{s}.fa", "r") as fr:
            for record in SeqIO.parse(fr, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])


## Reading data

In [ ]:
batch_size = 128

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 6791 files belonging to 2 classes.


In [ ]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

Found 750 files belonging to 2 classes.


In [ ]:
vectorize_layer = TextVectorization(output_mode='int')

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)

## Model training

In [ ]:
model_lstm = tf.keras.Sequential([
    keras.layers.Embedding(batch_size, 150, input_length = 150),
    keras.layers.LSTM(128, return_sequences= True),
    keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    keras.layers.Dense(1, activation="sigmoid")]) 

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10
history = model_lstm.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
54/54 [==============================] - 4s 76ms/step - loss: 0.6418 - accuracy: 0.6155 - val_loss: 0.5252 - val_accuracy: 0.7560
Epoch 2/10
54/54 [==============================] - 3s 55ms/step - loss: 0.4892 - accuracy: 0.7766 - val_loss: 0.4562 - val_accuracy: 0.7907
Epoch 3/10
54/54 [==============================] - 3s 53ms/step - loss: 0.4560 - accuracy: 0.7897 - val_loss: 0.5072 - val_accuracy: 0.7720
Epoch 4/10
54/54 [==============================] - 3s 56ms/step - loss: 0.4586 - accuracy: 0.7986 - val_loss: 0.4484 - val_accuracy: 0.8000
Epoch 5/10
54/54 [==============================] - 3s 54ms/step - loss: 0.4238 - accuracy: 0.8127 - val_loss: 0.4385 - val_accuracy: 0.8093
Epoch 6/10
54/54 [==============================] - 3s 54ms/step - loss: 0.4279 - accuracy: 0.8131 - val_loss: 0.4274 - val_accuracy: 0.8133
Epoch 7/10
54/54 [==============================] - 3s 54ms/step - loss: 0.4050 - accuracy: 0.8240 - val_loss: 0.3776 - val_accuracy: 0.8360
Epoch 8/10
54

In [ ]:
model_cnn = tf.keras.Sequential([
    keras.layers.Embedding(batch_size, 150, input_length = 150),
    keras.layers.Conv1D(32, kernel_size=6, data_format='channels_last', activation='tanh'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(16, kernel_size=6, data_format='channels_last', activation='tanh'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(4, kernel_size=6, data_format='channels_last', activation='sigmoid'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(1, activation="sigmoid")
])

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10

history = model_cnn.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
54/54 [==============================] - 2s 44ms/step - loss: 0.5332 - accuracy: 0.7544 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 2/10
54/54 [==============================] - 2s 36ms/step - loss: 0.4721 - accuracy: 0.8149 - val_loss: 0.6825 - val_accuracy: 0.5000
Epoch 3/10
54/54 [==============================] - 2s 38ms/step - loss: 0.4220 - accuracy: 0.8482 - val_loss: 0.6144 - val_accuracy: 0.7573
Epoch 4/10
54/54 [==============================] - 2s 37ms/step - loss: 0.3751 - accuracy: 0.8689 - val_loss: 0.5705 - val_accuracy: 0.7573
Epoch 5/10
54/54 [==============================] - 2s 37ms/step - loss: 0.3309 - accuracy: 0.8919 - val_loss: 0.6122 - val_accuracy: 0.5867
Epoch 6/10
54/54 [==============================] - 2s 37ms/step - loss: 0.3067 - accuracy: 0.8956 - val_loss: 0.7180 - val_accuracy: 0.5347
Epoch 7/10
54/54 [==============================] - 2s 37ms/step - loss: 0.2640 - accuracy: 0.9134 - val_loss: 0.6756 - val_accuracy: 0.5867
Epoch 8/10
54